In [2]:
import pandas as pd
import random

I selected this dataset from Arlington county. It's every transaction at a parking meter in Arlington from the last 10 years.  It's way too big for my computer (8GB, 36MM records), so I'm taking a random 10% sample

I'm not including the raw data in my repo, it can be downloaded here: https://data.arlingtonva.us/dataset/148?tab=2 

In [3]:

park = pd.read_csv(
    'ParkingTransactions.txt'
    ,delimiter='|'
    ,skiprows=lambda i: i>0 and random.random() > 0.1
    )

/var/folders/tn/1tqsy6b16xl_cxd8g27h18y00000gp/T/ipykernel_51561/3694701589.py:1: DtypeWarning: Columns (2,17) have mixed types. Specify dtype option on import or set low_memory=False.
  park = pd.read_csv(


In [4]:
park

,ParkingTransactionKey,TransactionSourceCode,MeterId,ZoneNbr,StartDtm,EndDtm,TransactionAmt,PaymentTypeName,TransactionStatusCode,MeterManufacturerName,...,StatePlaneXCrd,StatePlaneYCrd,HandicapInd,TimeRestrictionDsc,MaxHoursCnt,MeterTypeDsc,DollarPerHourRate,ActiveStatusInd,ZoneSpaceCnt,MetroAreaName
0,15,IPS,QUH2808S,54018.0,2015-12-31 16:53:11,2015-12-31 16:59:11,0.15,Coins,NaN,IPS,...,1.188539e+07,6.992200e+06,True,MON - SAT 8AM - 8PM,1.0,Single,1.75,True,1.0,Shirlington
1,34,IPS,191108N,54520.0,2015-12-31 15:33:18,2015-12-31 15:53:18,0.50,Coins,NaN,IPS,...,1.188987e+07,7.012885e+06,False,MON-SAT 8AM-6PM,4.0,Double,1.50,False,5.0,Rosslyn
2,43,IPS,191106N,54520.0,2015-12-31 14:55:53,2015-12-31 15:05:53,0.25,Coins,NaN,IPS,...,1.188987e+07,7.012886e+06,False,MON-SAT 8AM-6PM,4.0,Double,1.50,False,5.0,Rosslyn
3,90,IPS,WIH3101N,54307.0,2015-12-31 12:18:43,2015-12-31 13:39:51,0.20,Coins,NaN,IPS,...,1.188265e+07,7.009159e+06,True,MON-SAT 8AM-6PM,4.0,Single,1.50,False,1.0,Clarendon
4,260,IPS,CRH1700S,54688.0,2016-12-31 14:31:15,2016-12-31 14:53:15,0.55,Coins,NaN,IPS,...,1.189600e+07,6.998884e+06,True,MON - SAT 8AM - 8PM,4.0,Single,1.50,False,1.0,CRYSTAL CITY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612166,36754909,IPS,WAY1503N,NaN,2024-06-24 12:18:10,2024-06-24 12:32:17,0.25,Coins,NaN,IPS,...,NaN,NaN,False,MON - SAT 8AM - 8PM,0.5,Single,1.75,True,NaN,NaN
3612167,36754946,IPS,NE901N,NaN,2024-06-24 11:57:08,2024-06-24 12:05:43,0.25,Coins,NaN,IPS,...,1.188024e+07,7.007766e+06,False,MON - SAT 8AM - 8PM,2.0,Double,1.75,True,NaN,NaN
3612168,36755015,IPS,93803N,NaN,2024-06-24 11:14:55,2024-06-24 13:14:46,0.10,Coins,NaN,IPS,...,1.188003e+07,7.007162e+06,False,MON - SAT 8AM - 8PM,2.0,Double,1.75,False,NaN,NaN
3612169,36755082,IPS,WR105S,NaN,2024-06-24 09:55:12,2024-06-24 10:11:12,0.20,Coins,NaN,IPS,...,1.188655e+07,7.004049e+06,False,MON - FRI 8AM - 8PM,12.0,Single,0.75,True,NaN,NaN


In [5]:
#Pull out only a few cols and rename them
park_selected_cols = park[["ParkingTransactionKey","StartDtm","TransactionAmt","PaymentTypeName","TransactionSourceCode"]]
park_selected_cols.columns = ["id","start","amt","tender","source"]

park_selected_cols

,id,start,amt,tender,source
0,15,2015-12-31 16:53:11,0.15,Coins,IPS
1,34,2015-12-31 15:33:18,0.50,Coins,IPS
2,43,2015-12-31 14:55:53,0.25,Coins,IPS
3,90,2015-12-31 12:18:43,0.20,Coins,IPS
4,260,2016-12-31 14:31:15,0.55,Coins,IPS
...,...,...,...,...,...
3612166,36754909,2024-06-24 12:18:10,0.25,Coins,IPS
3612167,36754946,2024-06-24 11:57:08,0.25,Coins,IPS
3612168,36755015,2024-06-24 11:14:55,0.10,Coins,IPS
3612169,36755082,2024-06-24 09:55:12,0.20,Coins,IPS


When I tried to set the datatypes, I got a bunch of errors parsing the date column. 
Here was my first attempt to filter out records that had malformed dates.

In [6]:
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
        return True
    except ValueError:
        return False

# Filter out invalid dates
valid_records = park_selected_cols[park_selected_cols['start'].apply(is_valid_date)]
invalid_records = park_selected_cols[~park_selected_cols['start'].apply(is_valid_date)]

print(f"Dropped {len(invalid_records)} rows due to invalid dates.")

Dropped 120634 rows due to invalid dates.


A good practice - I didn't throwaway the bad records, I copied them into a new dataframe so that I could inspect them

and look what I found!

In [7]:
invalid_records

,id,start,amt,tender,source
1766544,24498233,2021-03-06 19:40:43.210000000,0.60,MASTERCARD,ParkMobile
1766545,24498254,2021-03-06 19:36:10.847000000,0.70,MASTERCARD,ParkMobile
1766546,24498286,2021-03-06 19:27:37.067000000,0.95,AMEX,ParkMobile
1766549,24498488,2021-03-06 18:51:47.643000000,2.00,VISA,ParkMobile
1766557,24498600,2021-03-06 18:36:11.323000000,2.45,VISA,ParkMobile
...,...,...,...,...,...
3365879,26593631,2021-09-20 07:51:29.080000000,3.50,VISA,ParkMobile
3365880,26593633,2021-09-20 07:51:09.007000000,3.50,MASTERCARD,ParkMobile
3365881,26593652,2021-09-20 07:45:58.110000000,3.50,VISA,ParkMobile
3365882,26593702,2021-09-20 07:30:43.350000000,0.05,VISA,ParkMobile


I definitely don't want to lose all the ParkMobile data, so I just need to filter these records differently

In [9]:
park_named_cols = valid_records.rename(columns={"ParkingTransactionKey":"id","StartDtm":"start","TransactionAmt":"amt","PaymentTypeName":"tender"})

park_named_cols['id'] = park_named_cols['id'].astype(int)
park_named_cols['month'] = pd.to_datetime(park_named_cols['start']).dt.month
park_named_cols['day'] = pd.to_datetime(park_named_cols['start']).dt.day
park_named_cols['year'] = pd.to_datetime(park_named_cols['start']).dt.year
park_named_cols['amt'] = (park_named_cols['amt'] * 100).astype(int)
park_named_cols = park_named_cols.drop(columns=['start'])

park_clean = park_named_cols

In [10]:
invalid_records_named = invalid_records.rename(columns={"ParkingTransactionKey":"id","StartDtm":"start","TransactionAmt":"amt","PaymentTypeName":"tender"})

invalid_records_named['id'] = invalid_records_named['id'].astype(int)
invalid_records_named['month'] = pd.to_datetime(invalid_records_named['start']).dt.month
invalid_records_named['day'] = pd.to_datetime(invalid_records_named['start']).dt.day
invalid_records_named['year'] = pd.to_datetime(invalid_records_named['start']).dt.year
invalid_records_named['amt'] = (invalid_records_named['amt'] * 100).astype(int)
invalid_records_named = invalid_records_named.drop(columns=['start'])

invalid_clean = invalid_records_named

In [12]:
#ok, put these two together, and select only certain cols

all_clean = pd.concat([park_clean,invalid_clean])

all_clean

,id,amt,tender,source,month,day,year
0,15,15,Coins,IPS,12,31,2015
1,34,50,Coins,IPS,12,31,2015
2,43,25,Coins,IPS,12,31,2015
3,90,20,Coins,IPS,12,31,2015
4,260,55,Coins,IPS,12,31,2016
...,...,...,...,...,...,...,...
3365879,26593631,350,VISA,ParkMobile,9,20,2021
3365880,26593633,350,MASTERCARD,ParkMobile,9,20,2021
3365881,26593652,350,VISA,ParkMobile,9,20,2021
3365882,26593702,5,VISA,ParkMobile,9,20,2021


Let's take a look at the summary stats for the numeric columns.

Pandas was being annoying about displaying the numbers in scientific notation. This lambda function uses an fstring to convert them to whole numbers.

In [13]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
all_clean.describe()

,id,amt,month,day,year
count,3612171,3612171,3612171,3612171,3612171
mean,18306214,224,7,16,2019
std,10583858,189,3,9,3
min,2,1,1,1,2014
25%,9158769,100,4,8,2017
50%,18252352,175,6,16,2019
75%,27443904,300,9,23,2021
max,36755177,14994,12,31,2024
